In [ ]:
import sys
sys.path.append("..")

from src.utils.pgsql import pg_df
import pandas as pd
import numpy as np
import json

In [ ]:
# import league specific scoring rules
rules_df = pg_df(
    "SELECT scoring_settings FROM leagues WHERE id = '1260107285024219136'"
    )

# If it's a string, parse it as JSON
if isinstance(rules_df, str):
    scoring_dict = json.loads(rules_df)
else:
    scoring_dict = rules_df

for rule, points in scoring_dict.items():
    print(f"{rule}: {points} points")

scoring_settings: 0    {'sack': 1.0, 'fgm_40_49': 4.0, 'pass_int': -2...
Name: scoring_settings, dtype: object points


In [26]:
# analysing seasonal data looking at prior seasons correlation to the following season
weekly_df = pg_df("""
    SELECT * 
    FROM nfl_weekly_stats 
    WHERE season >= 2010
    AND season_type = 'REG'
    """)

# create seasonal aggregation
season_cols = [
'player_id', 'season', 'player_name',
'position', 'position_group', 'recent_team_old', 

       'completions', 'attempts', 
       'passing_yards','passing_tds', 'interceptions', 
       'passing_first_downs', 'passing_2pt_conversions',

       'carries', 'rushing_yards', 
       'rushing_tds', 'rushing_fumbles',
       'rushing_fumbles_lost', 'rushing_first_downs', 
       'rushing_2pt_conversions', 

       'targets', 'receptions', 
       'receiving_yards', 'receiving_tds', 
       'receiving_fumbles', 'receiving_fumbles_lost',
       'receiving_first_downs', 'receiving_2pt_conversions'

]

# weekly to seasonal aggregation
season_df = weekly_df[season_cols].groupby(['player_id', 'season', 'player_name', 'position', 'position_group', 'recent_team_old']).agg('sum').reset_index()

# creating a next_season_point
# season_df['next_season_point'] = season_df['season'].shift(-1)

# creating a correlation matrix
# correlation_matrix = season_df.corr()
# season_df.columns
season_df.head()


,player_id,season,player_name,position,position_group,recent_team_old,completions,attempts,passing_yards,passing_tds,interceptions,passing_first_downs,passing_2pt_conversions,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_2pt_conversions,targets,receptions,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_first_downs,receiving_2pt_conversions
0,00-0023459,2024,A.Rodgers,QB,QB,NYJ,368.0,584.0,3897.0,28.0,11.0,192.0,2.0,22.0,107.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00-0024243,2024,M.Lewis,TE,TE,CHI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0
2,00-0026158,2024,J.Flacco,QB,QB,IND,162.0,248.0,1761.0,12.0,7.0,87.0,0.0,9.0,26.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00-0026300,2024,J.Johnson,QB,QB,BAL,2.0,3.0,17.0,0.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00-0026498,2024,M.Stafford,QB,QB,LA,340.0,517.0,3762.0,20.0,8.0,176.0,0.0,30.0,41.0,0.0,2.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
